# Ancient Greek Latent Space
Word2vec of lemmatized ancient greek - MIT Licence - 2023

In [34]:
from ipywidgets import HTML,Text,HBox,VBox,Button,Dropdown
from IPython.display import display,Javascript
from gensim.models import Word2Vec
import base64
model = Word2Vec.load("grf8_300.vec")

In [35]:
display(HTML('''
<style>
.tbl {width:80%}
lem {cursor:pointer;}
</style>

'''))


HTML(value='\n<style>\n.tbl {width:80%}\nlem {cursor:pointer;}\n</style>\n\n')

In [36]:
Javascript("""
function bailly(w){mot=w.innerHTML;window.open('https://bailly.app/q='+mot, '_blank').focus();}

""")

<IPython.core.display.Javascript object>

In [37]:
def expl(c):
    W.value='πόνος'
    gosim(c)

In [38]:
import glob
mchoice=Dropdown(options=glob.glob('*.vec'), description="Model size", value='grf8_300.vec')

def on_change(change):
    global model
    if change['type'] == 'change' and change['name'] == 'value':
        model=Word2Vec.load(change['new'])

mchoice.observe(on_change)



In [43]:
def link(w='πόνος',level=2):

    if level==0:
        return 1
    else:      
        r=model.wv.most_similar(w, topn=5)
        return [[f"{w}->{i}",link(w=i,level=level-1)] for i,_ in r]

graph=""
nodes={}
it=0
idx=0
def grap(c):
    global nodes,it,graph,idx
    mot=W.value
    graph=""
    nodes={}
    it=0
    idx=0

    def visit_nodes(x):
        global nodes,it,graph,idx
        if type(x) != list:
            if x!=1:
                n=x.split('->')
                for nn in n:
                    if nn not in nodes:
                        nodes[nn]=f"N{it}"
                        it+=1

                graph+=f'{{id:"{idx}", from:"{nodes[n[0]]}", to:"{nodes[n[1]]}"}},'
                idx+=1
            return

        for element in x:
            visit_nodes(element)

    
    visit_nodes(link(w=mot,level=3))
    nodesi={j:i for i,j in nodes.items()}


    total=""
    for j,i in nodes.items():
        total+=f'{{id:"{i}", label:"{j}"}},'
    template="""
    <head>
    <script src="https://visjs.github.io/vis-network/standalone/umd/vis-network.min.js"></script>

     <script type="text/javascript">
          var nodes, edges, network;



         function draw() {
            // create an array with nodes
            nodes = new vis.DataSet();

            nodes.add([@nodes@
            ]);

            // create an array with edges
            edges = new vis.DataSet();

            edges.add([@edges@        
            ]);

            // create a network
            var container = document.getElementById("mynetwork");
            var data = {
              nodes: nodes,
              edges: edges,
            };
            var options = {};
            network = new vis.Network(container, data, options);
          }

        </script>
    </head>

    <body onload="draw()">

    <div id="mynetwork" style="width:100%;height:800px"><div>	


    </body>"""

    template=template.replace("@nodes@",total[:-1])
    template=template.replace("@edges@",graph[:-1])
    url="data:text/html;base64,"+base64.b64encode(template.encode("utf-16")).decode()
    
    outp=f'<a href="{url}" download="{mot}.html" style="text-align:right;padding-right:10px;float:right;">Download</a>'
    outp+=f'<iframe src="{url}" style="width:100%;height:1200px" frameborder="0" scrolling="no"></iframe>'
    Out.value=outp



In [44]:
def gosim(c):
    try:
        r=model.wv.most_similar(W.value, topn=20)
        output="<table class='tbl'>"
        for m,s in r:
            output+=f"<tr><td><lem onclick='bailly(this)'>{m}</lem></td><td>{s:.2f}</td></tr>"
        output+='</table>'
    except:
        output="Unknown word"
    Out.value=output

In [41]:
W=Text(description="Word in greek")
B=Button(description="Similarity")
Be=Button(description="Example")
Bg=Button(description="Graph")
B.on_click(gosim)
Be.on_click(expl)
Bg.on_click(grap)
Out=HTML("")


In [42]:
def infer(c):
    try:
        r=model.wv.most_similar(positive=positive.value.split(' '), negative=negative.value.split(' '))
        output="<table class='tbl'>"
        for m,s in r:
            output+=f"<tr><td><lem onclick='bailly(this)'>{m}</lem></td><td>{s:.2f}</td></tr>"
        output+='</table>'
    except:
        output="Some words are unknown"
    Out.value=output
    
def expl2(c):
    positive.value="πόνος γυνή"
    negative.value="ἀνήρ"
    infer(c)

    

In [31]:
positive=Text(description="Positive")
negative=Text(description="Negative")
BB=Button(description="Infer")
BBe=Button(description='example')
Out2=HTML("")
BB.on_click(infer)
BBe.on_click(expl2)


In [32]:
VBox([mchoice,HTML("<h1>Word similarity</h1>"),
      VBox([HBox([W,B,Be,Bg])]),
      HTML("<h1>Sense addition</h1>"),
      VBox([HBox([positive,negative]),HBox([BB,BBe]),Out])])